In [0]:
%sql
CREATE OR REPLACE VIEW transacoes_db.pix_baseline_metricas.relacao_pagadores_recebedores AS

-- ===================================================
-- 1. Base agregada por município, mês e ano
-- ===================================================
WITH base_municipios AS (
  SELECT
    t.Ano,
    t.Mes,
    t.Estado_Ibge AS cod_ibge_estado,
    t.Estado AS estado_nome,
    t.Sigla_Regiao AS sigla_regiao,
    t.Regiao AS regiao_nome,
    t.Municipio_Ibge AS cod_ibge_municipio,
    t.Municipio AS municipio_nome,

    -- Totais de transações (PF / PJ / Total)
    SUM(t.QT_PagadorPF) AS total_tx_pagador_pf,
    SUM(t.QT_RecebedorPF) AS total_tx_recebedor_pf,
    SUM(t.QT_PagadorPJ) AS total_tx_pagador_pj,
    SUM(t.QT_RecebedorPJ) AS total_tx_recebedor_pj,

    SUM(t.QT_PagadorPF) + SUM(t.QT_PagadorPJ) AS total_tx_pagador,
    SUM(t.QT_RecebedorPF) + SUM(t.QT_RecebedorPJ) AS total_tx_recebedor,

    -- Totais de valor (PF / PJ / Total)
    SUM(t.VL_PagadorPF) AS total_vl_pagador_pf,
    SUM(t.VL_RecebedorPF) AS total_vl_recebedor_pf,
    SUM(t.VL_PagadorPJ) AS total_vl_pagador_pj,
    SUM(t.VL_RecebedorPJ) AS total_vl_recebedor_pj,

    SUM(t.VL_PagadorPF) + SUM(t.VL_PagadorPJ) AS total_vl_pagador,
    SUM(t.VL_RecebedorPF) + SUM(t.VL_RecebedorPJ) AS total_vl_recebedor,

    -- Razões (Pagador / Recebedor)
    CASE WHEN SUM(t.QT_RecebedorPF) = 0 THEN NULL
         ELSE ROUND(SUM(t.QT_PagadorPF) / SUM(t.QT_RecebedorPF), 4) END AS razao_pf,
    CASE WHEN SUM(t.QT_RecebedorPJ) = 0 THEN NULL
         ELSE ROUND(SUM(t.QT_PagadorPJ) / SUM(t.QT_RecebedorPJ), 4) END AS razao_pj,
    CASE WHEN (SUM(t.QT_RecebedorPF) + SUM(t.QT_RecebedorPJ)) = 0 THEN NULL
         ELSE ROUND(
            (SUM(t.QT_PagadorPF) + SUM(t.QT_PagadorPJ)) /
            (SUM(t.QT_RecebedorPF) + SUM(t.QT_RecebedorPJ)), 4) END AS razao_total

  FROM estatisticas_pix.gold.transacoes_municipio_pix t
  WHERE t.Municipio_Ibge IS NOT NULL
  GROUP BY t.Ano, t.Mes, t.Estado_Ibge, t.Estado, t.Sigla_Regiao, t.Regiao, t.Municipio_Ibge, t.Municipio
),

-- ===================================================
-- 2. Métricas derivadas e temporais
-- ===================================================
metrificacoes AS (
  SELECT
    bm.*,

    -- Proporções internas PF / PJ
    ROUND(bm.total_tx_pagador_pf / NULLIF(bm.total_tx_pagador, 0), 4) AS perc_pagador_pf,
    ROUND(bm.total_tx_pagador_pj / NULLIF(bm.total_tx_pagador, 0), 4) AS perc_pagador_pj,
    ROUND(bm.total_tx_recebedor_pf / NULLIF(bm.total_tx_recebedor, 0), 4) AS perc_recebedor_pf,
    ROUND(bm.total_tx_recebedor_pj / NULLIF(bm.total_tx_recebedor, 0), 4) AS perc_recebedor_pj,

    -- Variação temporal mês a mês (LAG)
    LAG(bm.total_tx_pagador) OVER (PARTITION BY bm.cod_ibge_municipio ORDER BY bm.Ano, bm.Mes) AS total_tx_pagador_mes_anterior,
    LAG(bm.total_tx_recebedor) OVER (PARTITION BY bm.cod_ibge_municipio ORDER BY bm.Ano, bm.Mes) AS total_tx_recebedor_mes_anterior,

    ROUND(
      (bm.total_tx_pagador - LAG(bm.total_tx_pagador) OVER (PARTITION BY bm.cod_ibge_municipio ORDER BY bm.Ano, bm.Mes)) /
      NULLIF(LAG(bm.total_tx_pagador) OVER (PARTITION BY bm.cod_ibge_municipio ORDER BY bm.Ano, bm.Mes), 0),
    4) AS taxa_crescimento_pagador,

    ROUND(
      (bm.total_tx_recebedor - LAG(bm.total_tx_recebedor) OVER (PARTITION BY bm.cod_ibge_municipio ORDER BY bm.Ano, bm.Mes)) /
      NULLIF(LAG(bm.total_tx_recebedor) OVER (PARTITION BY bm.cod_ibge_municipio ORDER BY bm.Ano, bm.Mes), 0),
    4) AS taxa_crescimento_recebedor
  FROM base_municipios bm
)

-- ===================================================
-- 3. Agregações avançadas e ranking
-- ===================================================
SELECT
  m.*,

  -- Ranking municipal por volume de pagadores e recebedores
  DENSE_RANK() OVER (PARTITION BY m.Ano, m.Mes ORDER BY m.total_tx_pagador DESC) AS rank_pagador_municipio,
  DENSE_RANK() OVER (PARTITION BY m.Ano, m.Mes ORDER BY m.total_tx_recebedor DESC) AS rank_recebedor_municipio,

  -- Participação relativa no total nacional
  ROUND(m.total_tx_pagador / NULLIF(SUM(m.total_tx_pagador) OVER (PARTITION BY m.Ano, m.Mes), 0), 6) AS peso_pagador_nacional,
  ROUND(m.total_tx_recebedor / NULLIF(SUM(m.total_tx_recebedor) OVER (PARTITION BY m.Ano, m.Mes), 0), 6) AS peso_recebedor_nacional

FROM metrificacoes m
ORDER BY rank_pagador_municipio, rank_recebedor_municipio, ano, mes
;